In [1]:
%pprint on
%pip install pip transformers gradio accelerate 'git+https://github.com/facebookresearch/fastText.git#main' 'txtai[all]' # --upgrade --force-reinstall --no-cache-dir

  Cloning https://github.com/facebookresearch/fastText.git to /private/var/folders/43/13h_2sgd3bxflq4m898ym8bm0000gn/T/pip-req-build-8zcovvqr
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git /private/var/folders/43/13h_2sgd3bxflq4m898ym8bm0000gn/T/pip-req-build-8zcovvqr
  Resolved https://github.com/facebookresearch/fastText.git to commit 6c2204ba66776b700095ff73e3e599a908ffd9c3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [2]:
!export CMAKE_ARGS="-DLLAMA_METAL=on"
%pip install llama-cpp-python # --no-binary  --upgrade --force-reinstall --no-cache-dir
%pip install 'llama-cpp-python[server]'

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#!python -m llama_cpp.server --model "$HOME/models/solar-10.7b-instruct-v1.0.Q5_K_M.gguf" --n_gpu_layers 1

In [4]:
from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor, LLM, Textractor
import nltk

if not nltk.downloader.Downloader().is_installed('punkt'):
    nltk.download('punkt')

In [5]:
#phi2_llm = LLM("TheBloke/phi-2-GGUF/phi-2.Q5_K_M.gguf", verbose=True)

In [6]:
solar_llm = LLM("TheBloke/SOLAR-10.7B-Instruct-v1.0-GGUF/solar-10.7b-instruct-v1.0.Q5_K_M.gguf", verbose=True, template="""<|im_start|>### System:
    You are a friendly assistant. You answer questions from users.<|im_end|>
    <|im_start|>### User:
    {text}
    <|im_end|>
    <|im_start|>### Assistant:
    """, n_ctx=4092, n_gpu_layers=1)

llama_model_loader: loaded meta data with 24 key-value pairs and 435 tensors from /Users/wangjuntham/.cache/huggingface/hub/models--TheBloke--SOLAR-10.7B-Instruct-v1.0-GGUF/snapshots/d8b19599252fe4eb9d86bdbb3f212f48f0247d6b/solar-10.7b-instruct-v1.0.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 48
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   

In [7]:
def prompt(query: str, question: str) -> list:
    return [{
        "query": query,
        "question": question
    }]

In [8]:
def stream():
    textract = Textractor(paragraphs=True)
    for thing in textract("zarathustra.md"):
        yield thing.strip().replace("\n", "")

embeddings = Embeddings(content=True, autoid="uuid5", path='sentence-transformers/all-MiniLM-L12-v2', gpu=True)
# embeddings.index(stream())
# embeddings.save("zarathustra.index.tar.gz")
embeddings.load("zarathustra.index.tar.gz")

In [9]:
extractor = Extractor(embeddings, solar_llm, output="reference", context=48, mintokens=64, template="""
        From the matching text in the given context, answer my question in great detail. The response should be related to the text from the context only.
        
        #### Question:
        {question}
        
        #### Context:
        {context}
        """)

In [10]:
# question = "What does Zarathustra have to say about pity?"

In [17]:
question = input()
print(question)
result = extractor(prompt(question, question), maxlength=4096)
print(result[0]["answer"])
print(embeddings.search("select id, text from txtai where id = :id", limit=48, parameters={"id": result[0]["reference"]}))

How are creators described and interpreted?


Llama.generate: prefix-match hit


In the given context, creators are described as individuals who seek to establish new values and redefine existing ones. They break old laws and value systems, often being perceived as law-breakers or despisers of good and evil by society. However, they are also referred to as harvesters and rejoicers, playing a crucial role in shaping the meaning and future of humanity. Creators are compared to mothers, enduring pain and willingness to destroy old structures for the emergence of something new. They are differentiated from actors and fabricators as their primary motivation is creation rather than presentation or manipulation. This process of creating often involves evaluating what already exists, thereby giving meaning to their actions. Creators might be seen as complex, hard, and sometimes unclean due to their transformative nature, but they possess the power to create new values and freedom for continuous innovation.
[{'id': '6b006209-6c0b-5143-9d09-047fd8203bab', 'text': 'A change i


llama_print_timings:        load time =    3367.66 ms
llama_print_timings:      sample time =      18.83 ms /   184 runs   (    0.10 ms per token,  9773.20 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   13903.31 ms /   184 runs   (   75.56 ms per token,    13.23 tokens per second)
llama_print_timings:       total time =   14204.12 ms
